# Visualization with py3Dmol (optional)

pyvoro2 includes a small **optional** helper module, `pyvoro2.viz3d`, for interactive visualization
in notebooks. It is meant for exploration and debugging:

- draw the domain wireframe (box or unit cell),
- draw cell wireframes from the `vertices`/`faces` output,
- optionally draw labeled sites and (global) vertices.

Install the extra dependency with:

```bash
pip install "pyvoro2[viz]"
```

or directly:

```bash
pip install py3Dmol
```

> These helpers are intentionally lightweight. For publication-quality rendering you will usually
> want a dedicated 3D pipeline.


In [1]:
import numpy as np
import pyvoro2 as pv

from pyvoro2.viz3d import VizStyle, view_tessellation

rng = np.random.default_rng(0)


## 1) Non-periodic box

In a non-periodic `Box`, all returned vertices are inside the domain boundary.


In [2]:
points = rng.uniform(-1.0, 1.0, size=(15, 3))
box = pv.Box(((-2, 2), (-2, 2), (-2, 2)))

cells = pv.compute(
    points,
    domain=box,
    return_vertices=True,
    return_faces=True,
)

view_tessellation(
    cells,
    domain=box,
    show_vertices=False,  # start simple
)


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

## 2) Styling with VizStyle

All visual parameters (radii, line widths, label sizes, colors) are collected in a small dataclass
`VizStyle`. You can pass a modified style object to `view_tessellation(...)`.

Below we:
- make sites smaller,
- make edges thicker,
- and show vertex markers.


In [3]:
style = VizStyle(
    site_radius=0.06,
    edge_line_width=4.0,
    vertex_radius=0.03,
    site_label_font_size=7,
    vertex_label_font_size=6,
)

view_tessellation(
    cells,
    domain=box,
    style=style,
    show_vertices=True,
    show_vertex_labels='off',
)


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

## 3) Periodic triclinic unit cell: vertices may lie outside the primary cell

In periodic tessellations it is normal for some cell vertices to lie outside the “primary” unit cell.
This does **not** mean the tessellation is wrong: it is a consequence of representing a periodic
polyhedron in Cartesian space.

In the next cell we show a random configuration in a tilted triclinic cell.


In [4]:
cell = pv.PeriodicCell(
    vectors=((10.0, 0.0, 0.0), (2.0, 9.0, 0.0), (1.0, 0.5, 8.0)),
    origin=(0.0, 0.0, 0.0),
)

points_p = rng.random((20, 3))  # arbitrary Cartesian points

cells_p = pv.compute(
    points_p,
    domain=cell,
    return_vertices=True,
    return_faces=True,
    return_face_shifts=True,
)

view_tessellation(
    cells_p,
    domain=cell,
    show_vertices=False,
)


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

## 4) Wrapping cells for visualization

If `wrap_cells=True`, each cell geometry is translated by an integer lattice vector so that its
site lies inside the primary unit-cell parallelepiped. This is purely a visualization convenience.


In [5]:
view_tessellation(
    cells_p,
    domain=cell,
    wrap_cells=True,
    show_vertices=False,
)


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

## 5) Global vertex labels via normalize_vertices / normalize_topology

If you normalize vertices/topology, the view can show **global vertex ids** (\`v0\`, \`v1\`, ...).
This is useful when you want to compare connectivity across cells or debug periodic graphs.

In periodic tessellations the same global vertex can appear in several periodic images.
The label always refers to the **global id**, while the marker position follows the drawn
wireframe geometry for the currently shown cells.


In [6]:
from pyvoro2 import normalize_vertices

nv = normalize_vertices(cells_p, domain=cell)

view_tessellation(
    nv,
    domain=cell,
    wrap_cells=False,
    show_vertices=True,
    show_vertex_labels='auto',  # show labels only when feasible
)


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

## Tips

- For large systems, visualize a subset of cells (`cell_ids={0, 1, 2}`) or only sites (`show_vertices=False`).
- If labels clutter the view, disable them (`show_site_labels=False`) or reduce `max_site_labels`.
- The visualization helpers accept the outputs of `ghost_cells(...)` as well, which is handy for probe analysis.
